In [1]:
#import all libraries
import pandas as pd 
import numpy as np 
import snscrape.modules.twitter as sntwitter 
import datetime 
from tqdm.notebook import tqdm_notebook 
import unicodedata
import re
import contractions
import string

In [3]:
#get input variables
text = input('Enter query text to be matched (or leave it blank by pressing enter)') 
since = input('Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 
until = input('Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 
count = int(input('Enter max number of tweets or enter -1 to retrieve all possible tweets: ')) 
retweet = input('Exclude Retweets? (y/n): ') 
replies = input('Exclude Replies? (y/n): ') 

Enter query text to be matched (or leave it blank by pressing enter)Bangladesh
Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): 2023-01-01
Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): 2023-03-31
Enter max number of tweets or enter -1 to retrieve all possible tweets: 25000
Exclude Retweets? (y/n): y
Exclude Replies? (y/n): y


In [4]:
def search(text,since,until,retweet,replies):
    global filename
    q = text    
    if until=='':
        until = datetime.datetime.strftime(datetime.date.today(), '%Y-%m-%d')
    q += f" until:{until}"
    if since=='':
        since = datetime.datetime.strftime(datetime.datetime.strptime(until, '%Y-%m-%d') - 
                                           datetime.timedelta(days=7), '%Y-%m-%d')
    q += f" since:{since}"
    if retweet == 'y':
        q += f" exclude:retweets"
    if replies == 'y':
        q += f" exclude:replies"
    
    return q

In [ ]:
q = search(text,since,until,retweet,replies)
tweets_list1 = []
if count == -1:
    for i,tweet in enumerate(tqdm_notebook(sntwitter.TwitterSearchScraper(q).get_items())):
        tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.lang,
        tweet.hashtags,tweet.replyCount,tweet.retweetCount,tweet.likeCount,tweet.quoteCount,tweet.media])
else:
    with tqdm_notebook(total=count) as pbar:
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()): #declare a username 
            if i>=count: #number of tweets you want to scrape
                break
            tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.lang,
        tweet.hashtags,tweet.replyCount,tweet.retweetCount,tweet.likeCount,tweet.quoteCount,tweet.media])
            pbar.update(1)
# Creating a dataframe from the tweets list above 
tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username','Language',
                                'Hashtags','ReplyCount','RetweetCount','LikeCount','QuoteCount','Media'])